In [271]:
# https://bigbuild.disruptions-app.com.au/api/disruptions/?ordering=start_date&search=Gippsland&type[]=18
# https://bigbuild.disruptions-app.com.au/api/disruptions/?ordering=start_date&search=Traralgon&type[]=18
# https://bigbuild.disruptions-app.com.au/api/disruptions/?ordering=start_date&search=Bairnsdale&type[]=18

# Type 18 is V/Line Disruptions

import datetime
from dateutil import parser
import requests
import icalendar
from pathlib import Path
import json
import re
import html2text

htmlConverter = html2text.HTML2Text()
htmlConverter.ignore_links = True
htmlConverter.ignore_emphasis = True

# A function that returns the 'year' value:
def myFunc(e):
  return e['start']

def cleanHtml(text):
    return re.sub(r"(?:(?!\n)\s)+", " ", 
                  htmlConverter.handle(text)
                  .replace("\\", '')
                  .replace("\n", ' ')
            ).strip()

def removeLinks(text):
    return re.sub(r"((For more information|Check the|visit the|--- Event details:|allow|See details|For further).*)|please", "", text,flags=re.IGNORECASE).strip() 

# on Victoria's Big Build visit https://bigbuild.vic.gov.au/disruptions", '').replace(r"For more information visit https://bigbuild.vic.gov.au/disruptions/disruptions-map",'')


# SUMMARY = name
# DTSTART = start_date
# DTEND = end_date
# X-ALT-DESC (DESCRIPTION) = description
# LOCATION = location

names = ['gippsland', 'traralgon', 'bairnsdale']
updates = {}
datetimeformat = "%Y-%m-%d %H:%M:%S%z"
# datetimeformat = "%Y-%m-%d"
location = []
disruptionString = r'(\?|&)disruption=(<br>)?[\d]+'

for name in names:
    file = json.load(open(f'{name}.json', "r"))
    for event in file:
        if event["id"] not in updates:
            update = {}
            update["summary"] = event.get("name") #.replace('Gippsland Line - ', '').replace('Gippsland Line')
            update["location"] = event.get("location")
            update["project"] = int(re.search(r'[\d]+', event.get("project")).group())
            update["description"] = cleanHtml(event["description"]) # .replace("<p>", '').replace('</p>', '')
            update["start"] = parser.parse(event["start_date"]).strftime(datetimeformat)
            update["end"] = parser.parse(event["end_date"]).strftime(datetimeformat)
            update["id"] = event["id"]
            update["description"] = removeLinks(update["description"])

            updates[update["id"]] = update
        

i = 1
ics_path = Path('disruption-calendar.ics')
calendar = icalendar.Calendar.from_ical(ics_path.read_bytes())
for event in calendar.events:
    update = {}
    update["summary"] = str(event.get("SUMMARY"))
    update["location"] = str(event.get("LOCATION", None))
    description = event["X-ALT-DESC"]
    update["description"] = removeLinks(cleanHtml(description))
    update["start"] = event.DTSTART.strftime(datetimeformat)
    update["end"] = event.DTEND.strftime(datetimeformat)
    id = re.search(disruptionString, description)

    if id is not None: 
        update["id"] = int(re.search(r'[\d]+', id.group()).group())
    else:
        addevent = re.search(r'https://www.addevent.com/event/[0-9A-z]+', description)
        if addevent is not None:
            event = requests.get(addevent.group()).text
            id = re.search(disruptionString, event)
    
            # Not worth it. All the Add Event Disruptions are well documented, and we'd need to clean this up.
            # update["description"] = cleanHtml(event)
            # print(update["description"])

            if id is not None:
                update["id"] = int(re.search(r'[\d]+', id.group()).group())
            else:
                # print(event)
                # print(addevent.group())
                # print(addevent.group())
                update["id"] = 'Unknowndv' + str(i)
                i = i + 1
        else:
            update["id"] = 'Unknown' + str(i)
            print(description)
            i = i + 1

    if update["id"] in updates:
        # print(update["id"])
        existUpdate = updates[update["id"]]
        if len(existUpdate["summary"]) < len(update["summary"]):
            existUpdate["summary"] = update["summary"]
        if len(existUpdate["description"]) < len(update["description"]):
            existUpdate["description"] = update["description"]
    else:
        updates[update["id"]] = update

    # updates.append(update)
    # else:
    #     print(update)
    #     i = i+1


# len(updates)
# updates.sort(key=myFunc)

updates
# location

Coaches replace trains between Sale and Bairnsdale stations on Sunday 29 November.
5314
Coaches replace trains between Sale and Bairnsdale stations from Thursday 3 December to last service on Sunday 6 December.
5874
5874
6094
6345
6384
6447
6485
6567
6645
6665
6692
6694
6777
6852
6893
6892
6939
6935
6937
6989
6976
7032
6836
7212
7165
7262
7204
7396
7414
7684
7660
7662
7709
7788
7797
7811
7944
8116
8131
8296
8297
8281
8298
8492
8494
8576
8578
8580
8281
8590
8875
8888
8742
8917
Coaches replace some trains between Southern Cross and Pakenham/Traralgon stations from Sunday 2 October to Wednesday 12 October. 
8746
Coaches replace trains between Southern Cross and Pakenham stations on Sunday 20 November.<br><br>For more information visit
Coaches replace evening trains between Southern Cross and Pakenham stations from Monday 28 November to Thursday 1 December. 
9547
9768
9846
Coaches replace trains between Southern Cross, Traralgon and Bairnsdale from Saturday 4 February to last train, Sunday

{2636: {'summary': 'Gippsland line – coaches replacing trains',
  'location': 'Between Southern Cross, Traralgon and Bairnsdale stations',
  'project': 16,
  'description': 'Coaches replacing all Traralgon and Bairnsdale trains for the entire journey from Saturday 1 December to Sunday 13 January.',
  'start': '2018-12-01 01:00:00+1100',
  'end': '2019-01-14 00:59:00+1100',
  'id': 2636},
 3228: {'summary': 'Gippsland line - coaches replace trains',
  'location': 'Between Southern Cross, Traralgon and Bairnsdale',
  'project': 29,
  'description': 'Coaches replace trains from the evening of Friday 22 March to Sunday 24 March.',
  'start': '2019-03-22 01:00:00+1100',
  'end': '2019-03-25 00:59:00+1100',
  'id': 3228},
 3282: {'summary': 'Gippsland line - coaches replace trains',
  'location': 'Between Southern Cross,Traralgon and Bairnsdale',
  'project': 29,
  'description': 'Coaches replace trains from Saturday 6 April to the last service on Tuesday 23 April.',
  'start': '2019-04-06 0

In [2]:
hasattr('check', 'check')

False

In [272]:
json_output = json.dumps(updates)
with open(f'updates.json', "w") as f:
    f.write(json_output)